2017

Machine Learning Practical

University of Edinburgh

Georgios Pligoropoulos - s1687568

Coursework 4 (part 5a)

### Imports, Inits, and helper functions

In [1]:
jupyterNotebookEnabled = True
plotting = True
saving = True
coursework, part = 4, "5a"

if jupyterNotebookEnabled:
    #%load_ext autoreload
    %reload_ext autoreload
    %autoreload 2

In [2]:
import sys
import os
mlpdir = os.path.expanduser(
    '~/pligor.george@gmail.com/msc_Artificial_Intelligence/mlp_Machine_Learning_Practical/mlpractical'
)
sys.path.append(mlpdir)

In [3]:
import pickle
import skopt
from skopt.plots import plot_convergence
import datetime
import time
import tensorflow as tf
import numpy as np
    
import matplotlib.pyplot as plt

from mylibs.tf_helper import tfRMSE, tfMSE, fully_connected_layer, trainEpoch, validateEpoch

from mylibs.py_helper import merge_dicts
    
from mylibs.dropout_helper import constructProbs

from skopt.space.space import Real, Integer
from skopt import gp_minimize

from mylibs.jupyter_notebook_helper import show_graph

from mlp.data_providers import DataProvider, \
    MSD10GenreDataProvider, MSD25GenreDataProvider,\
    MSD10Genre_Autoencoder_DataProvider, MSD10Genre_StackedAutoEncoderDataProvider
    
from mylibs.batch_norm import batchNormWrapper_byExponentialMovingAvg, fully_connected_layer_with_batch_norm

from mylibs.batch_norm import fully_connected_layer_with_batch_norm_and_l2

from mylibs.stacked_autoencoder_pretrainer import \
    constructModelFromPretrainedByAutoEncoderStack,\
    buildGraphOfStackedAutoencoder, executeNonLinearAutoencoder
    
from mylibs.jupyter_notebook_helper import getRunTime, getTrainWriter, getValidWriter,\
    plotStats, initStats, gatherStats, renderStatsCollection

In [4]:
if jupyterNotebookEnabled:
    %matplotlib inline

seed = 16011984
rng = np.random.RandomState(seed=seed)

config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

figcount = 0

tensorboardLogdir = 'tf_cw{}_{}'.format(coursework, part)

curDtype = tf.float32

reluBias = 0.1

batch_size = 50
numTestSongs = 9950
numClasses = 10

### MSD 10 genre task

In [5]:
segment_count = 120
segment_len = 25

In [6]:
from models.teacher_student_nn import MSD10Genre_Teacher_DataProvider, StudentNN

In [7]:
from rnn.manual_rnn import ManualRNN

In [8]:
dataset_filename = 'msd-10-genre-train_valid.npz'

In [9]:
logits_filename = 'rnn_logits.npy'

### Validating with Teacher

In [11]:
manualRNN = ManualRNN(batch_size=batch_size, rng = rng, dtype = curDtype, config=config,
                      segment_count = segment_count, segment_len = segment_len)

In [12]:
best_params_filename = 'best_params_rnn.npy'
(state_size, num_steps) = np.load(best_params_filename)
(state_size, num_steps)

(341, 4)

In [13]:
data_provider = manualRNN.get_dp("train_valid", num_steps = num_steps)

In [14]:
graph = manualRNN.getGraph(num_steps=num_steps, state_size=state_size)

In [17]:
epochs = 20

In [18]:
%%time

stats, keys = manualRNN.validate(data_provider, state_size, graph, epochs=epochs, verbose=True)

epochs: 20
rnn steps: 4
state size: 341
End epoch 01 (46.634 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 02 (46.672 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 03 (46.347 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 04 (46.027 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 05 (45.852 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 06 (45.920 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 07 (45.909 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 08 (45.864 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 09 (46.181 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 10 (45.946 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 11 (46.052 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 12 (45.538 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 13 (45.458 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 14 (45.520 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 15 (45.849 secs): err(valid)=2.57, acc(valid)=0.12, 
End epoch 16 (4

In [1]:
times = [46.634,
46.672,
46.347,
46.027,
45.852,
45.920,
45.909,
45.864,
46.181,
45.946,
46.052,
45.538,
45.458,
45.520,
45.849,
45.888,
45.531,
45.583,
45.557,
45.947,]

In [5]:
np.mean(times)

45.913749999999993

In [6]:
np.min(times)

45.457999999999998

In [7]:
np.max(times)

46.671999999999997

In [8]:
np.var(times)

0.11426098749999994

### Running full training of shallow neural net

In [10]:
studentNN = StudentNN(batch_size=batch_size, rng=rng, dtype=curDtype, config=config)

learning_rate = "1e-4"

best_params_student_teacher_filename = 'student_teacher/{}/student_teacher_best_params.npy'.\
    format(learning_rate)
best_params_student_teacher = np.load(best_params_student_teacher_filename)
best_params_student_teacher

array([  5.00000000e-01,   1.00000000e+00,   1.78400000e+03,
         1.00000000e-03])

In [14]:
%%time

input_keep_prob, hidden_keep_prob, hidden_dim, lamda2 = best_params_student_teacher
hidden_dim = int(hidden_dim)
learning_rate = float(learning_rate)

graph = studentNN.loadAndGetGraph(hidden_dim=hidden_dim, lamda2=lamda2, learningRate=learning_rate)

data_provider = MSD10GenreDataProvider('train_valid', batch_size=batch_size, rng=rng)

epochs = 20

stats, keys, runTimes = studentNN.validate(
    data_provider = data_provider,
    graph = graph,
    epochs = epochs,
)

lamda2: 0.001000
hidden dim: 1784
learning rate: 0.000100
epochs: 20
End epoch 01 (9.714 secs): err(valid)=4.55, acc(valid)=0.10, 
End epoch 02 (9.365 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 03 (9.461 secs): err(valid)=4.47, acc(valid)=0.10, 
End epoch 04 (9.356 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 05 (9.382 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 06 (9.347 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 07 (9.345 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 08 (9.340 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 09 (9.347 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 10 (9.385 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 11 (9.324 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 12 (9.461 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 13 (9.304 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 14 (9.381 secs): err(valid)=4.48, acc(valid)=0.10, 
End epoch 15 (9.376 secs): err(valid)=4.48, acc(valid)=0.10, 
E

In [15]:
np.mean(runTimes)

9.4196129441261292

In [16]:
np.min(runTimes)

9.3039250373840332

In [17]:
np.max(runTimes)

10.014816045761108

In [18]:
np.var(runTimes)

0.02560755415333844